In [1]:
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time

import seaborn as sns
plt.style.use('seaborn')

In [2]:
segments = np.arange(10)

start = []
stop = []
step = 1
400

(40*4)/400

#tester.optimize_strategy((1,400,6),(1,400,6),(1,5,12), metric = "Multiple") #quick
#tester.optimize_strategy((205,400,50),(10,150,25),(1,2,6), metric = "Multiple") #quick

0.4

In [3]:
segments

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
start = 0
stop = 399
resultGroupSize = 75
segments = np.arange(stop/resultGroupSize)

#df = pd.DataFrame(#append each comb to a row

print('There are {} segments with {} results in each group.\n'.format(str(len(segments)), str(0)))

for i in range(len(segments)):
    if i < len(segments)-1:
        print(1*segments[i]+start+stop/len(segments)*segments[i] , ' to ' , str(stop/len(segments)*segments[i+1]))
        print('\nThen, \n')
    else:
        print(str(1+start+stop/len(segments)*segments[i]) + ' to ' + str(stop))



There are 6 segments with 0 results in each group.

0.0  to  66.5

Then, 

67.5  to  133.0

Then, 

135.0  to  199.5

Then, 

202.5  to  266.0

Then, 

270.0  to  332.5

Then, 

333.5 to 399
